--Comprobar entorno (hf_env)--

In [1]:
import sys
print(sys.executable)
import torch, transformers, datasets
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("Datasets:", datasets.__version__)

c:\Users\Pasi\miniconda3\envs\hf_env\python.exe


c:\Users\Pasi\miniconda3\envs\hf_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch: 2.8.0+cpu
Transformers: 4.56.1
Datasets: 4.1.0


-- Fijar una semilla para la reproducibilidad--

In [2]:
import random
import numpy as np
import torch

# Función para fijar la semilla en todas las librerías relevantes
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        
set_seed(12)  # Fijar una semilla para la reproducibilidad

--Cargar el dataset (TASS 2020)--

In [3]:
from datasets import load_dataset

# Cargamos el dataset
dataset = load_dataset("pysentimiento/spanish-targeted-sentiment-headlines")

print(f"Columnas del dataset: {dataset['train'].column_names}")

Columnas del dataset: ['titulo', 'id_noticia', 'target', 'label']


--Tokenizador y modelo base--

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Definimos el modelo a utilizar preentrenado en Hugging Face (RoBERTuito)
model_name = "pysentimiento/robertuito-sentiment-analysis"

# Cargamos el tokenizador y el modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)

--Tokenización--

In [10]:
# Función de tokenización
def tokenize_function(examples):
    # Tokenizamos los textos con padding y truncamiento
    return tokenizer(examples["titulo"], padding="max_length", truncation=True, max_length=tokenizer.model_max_length)

# Aplicamos la tokenización al conjunto de datos
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 459/459 [00:00<00:00, 7983.75 examples/s]


--Preparación del modelo para la clasificación--

In [6]:
# Comprobamos las etiquetas en el conjunto de datos
labels = dataset["train"].unique("label")
num_labels = len(labels)


# Mostramos el número de etiquetas y las etiquetas mismas
print(f"Número de Etiquetas: {num_labels}, Etiquetas: {labels}")

Número de Etiquetas: 3, Etiquetas: [2, 1, 0]


In [7]:
# Cargamos el modelo para clasificación de secuencias con el número adecuado de etiquetas
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

--Configuración del entrenamiento con la ayuda de Trainer--

In [8]:
from transformers import TrainingArguments, Trainer
import evaluate


accuracy = evaluate.load("accuracy") # Métrica de precisión global
f1 = evaluate.load("f1")             # Métrica F1 (media armónica de precisión y recall)
precision = evaluate.load("precision") # Métrica de precisión
recall = evaluate.load("recall")     # Métrica de Exhaustividad / Sensibilidad

# Función para computar las métricas durante la evaluación
def compute_metrics(eval_pred):
    # Separa las predicciones y las etiquetas verdaderas
    logits, labels = eval_pred
    # Obtiene las predicciones finales tomando el índice de la clase con mayor logit
    predictions = logits.argmax(axis=-1)
    # Calcula y devuelve la precisión
    return {
        "accuracy": accuracy.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1.compute(predictions=predictions, references=labels, average="macro")["f1"],
        "precision": precision.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "recall": recall.compute(predictions=predictions, references=labels, average="weighted")["recall"],
    }

In [13]:
# Configuración de los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",          # Directorio para guardar los resultados
    num_train_epochs=3,              # Número de épocas de entrenamiento 
    learning_rate=2e-5,              # Tasa de aprendizaje
    per_device_train_batch_size=16,  # Tamaño del batch para entrenamiento
    per_device_eval_batch_size=16,   # Tamaño del batch para evaluación
    weight_decay=0.01,               # Decaimiento del peso
    logging_dir="./logs",            # Directorio para los logs
    seed=12,                         # Semilla para reproducibilidad
    eval_strategy="steps",     # Estrategia de evaluación
    eval_steps=100,                  # Evaluar cada 100 pasos
    save_strategy="steps",           # Estrategia de guardado
    save_steps=100,                  # Guardar el modelo cada 100 pasos
    load_best_model_at_end=True,     # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="f1",      # Métrica para seleccionar el mejor modelo
    greater_is_better=True,          # Indica si una métrica mayor es mejor
)

In [14]:
from transformers import EarlyStoppingCallback

# Crear el entrenador
trainer = Trainer(
    model=model,                                                  # El modelo a entrenar
    args=training_args,                                           # Argumentos de entrenamiento
    train_dataset=tokenized_datasets["train"],                    # Conjunto de datos de entrenamiento
    eval_dataset=tokenized_datasets["test"],                      # Conjunto de datos de validación
    compute_metrics=compute_metrics,                              # Función para computar métricas
    tokenizer=tokenizer,                                          # Tokenizador
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)], # Callback para early stopping
)

C:\Users\Pasi\AppData\Local\Temp\ipykernel_14936\981696436.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


--Entrenamiento--

In [15]:
# Entrenar el modelo
trainer.train()

c:\Users\Pasi\miniconda3\envs\hf_env\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,No log,0.763048,0.688013,0.684870,0.691104,0.688013
200,No log,0.755383,0.701149,0.697437,0.702496,0.701149


c:\Users\Pasi\miniconda3\envs\hf_env\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=258, training_loss=0.6473154467205668, metrics={'train_runtime': 1178.0589, 'train_samples_per_second': 3.491, 'train_steps_per_second': 0.219, 'total_flos': 270546371778816.0, 'train_loss': 0.6473154467205668, 'epoch': 3.0})

--Guardar el modelo fine-tuned--

In [16]:
# Guardar el modelo y el tokenizador entrenados
trainer.save_model("./tass-sentiment-model")
tokenizer.save_pretrained("./tass-sentiment-model")

('./tass-sentiment-model\\tokenizer_config.json',
 './tass-sentiment-model\\special_tokens_map.json',
 './tass-sentiment-model\\tokenizer.json')

--Prueba del model fine-tuned--

In [17]:
from transformers import pipeline

classifier = pipeline("text-classification", model="./tass-sentiment-model", tokenizer="./tass-sentiment-model")

print(classifier("La película fue fantástica y me encantó cada momento."))
print(classifier("El servicio en el restaurante fue terrible y no volveré."))
print(classifier("No estuvo ni bien ni mal, fue una experiencia promedio."))

Device set to use cpu


[{'label': 'POS', 'score': 0.9845624566078186}]
[{'label': 'NEG', 'score': 0.9460989236831665}]
[{'label': 'POS', 'score': 0.9520020484924316}]


--Evaluación--

In [18]:
metrics = trainer.evaluate()
print(metrics)

c:\Users\Pasi\miniconda3\envs\hf_env\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.755383312702179, 'eval_accuracy': 0.7011494252873564, 'eval_f1': 0.6974367598733996, 'eval_precision': 0.7024959202774285, 'eval_recall': 0.7011494252873564, 'eval_runtime': 39.7271, 'eval_samples_per_second': 15.33, 'eval_steps_per_second': 0.982, 'epoch': 3.0}
